## First Model - Linear Regression

- __Group name__: PretendCoders
- __Author names__: 
- Rita Zhang  - zhang_hong@bentley.edu
- Anthony Arno  - arn1_anth@bentley.edu
- Vibutesh Ravisankar  - ravison_vibu@bentley.edu

In [2]:
%sh ls /dbfs/mnt/group-ma755/data/ais*

/dbfs/mnt/group-ma755/data/ais-test.csv
/dbfs/mnt/group-ma755/data/ais-train.csv

In [3]:
%run "/Courses/MA755/Groups/pretendcoders/Final Notebooks/A3.0 - Feature Engineering"

Import `numpy`,`pandas`,`sklearn`. Note the corresponding versions.

In [5]:
import numpy             as np
import pandas            as pd
import sklearn as sk


from pandas import Series
from pandas import DataFrame
from sklearn import pipeline
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import gen_features
import sklearn.preprocessing, sklearn.decomposition, \
       sklearn.linear_model,  sklearn.pipeline, \
       sklearn.metrics
from sklearn.model_selection import GridSearchCV

np.__version__, pd.__version__, sk.__version__

Out[ 19 ]: ('1.14.3', '0.18.1', '0.18.1')

Create the training target value `import_t_y`

In [7]:
import_t_y = np.array(ais_t_df.Average.shift(-5))
where_nan=np.isnan(import_t_y)
import_t_y[where_nan]=0

In [8]:
import_test_y = np.array(ais_testnew_df.Average.shift(-5))
where_nan1=np.isnan(import_test_y)
import_test_y[where_nan1]=0

In [9]:
from sklearn import pipeline
from sklearn.linear_model  import LinearRegression
binarizer_list=[['Month'],['Day']]
external_data_list=[['DJIA'],['Crude Oil Price'],['Dollar Index'],['Indicator']]
mlpipeline = pipeline.Pipeline([('Date', DateTransformer()),
                               ('Unique', UniqueTransformer()),
                               ('AppendYearMonthDay', AddDatesTransformer()),
                               ('MergeFeatures', MergeFeaturesTransformer()),
                               ('MergeFeatures2', MergeFeaturesTransformer(df_name=CrudeOil_df)),
                               ('MergeFeatures3', MergeFeaturesTransformer(df_name=DollarIndex_df)), 
                               ('MergeFeatures4', MergeFeaturesTransformer(df_name=China_Recession_Binary_df)),   
                               ('Type', TypeTransformer(col_name='DJIA')),
                               ('Type2', TypeTransformer(col_name='Crude Oil Price')),
                               ('Type3', TypeTransformer(col_name='Dollar Index')),
                               
                               ('feature', FeatureUnion([('lags',mapper_lags(n_lags=75,col_name='Average')),
                                                         ('class', DataFrameMapper( 
                                                                   gen_features(columns=binarizer_list,
                                                                   classes=[sklearn.preprocessing.LabelBinarizer]),
                                                                   input_df=True)),
                                                         ('new_data', DataFrameMapper(
                                                                     gen_features(columns=external_data_list,
                                                                     classes=None),input_df=True)),
                                                         ('windows',mapper_window(col_name='Average')),
                                                         ('windows2',mapper_window(col_name='DJIA'))
                                                                             ])),
                               ('align',  NaNTransformer()),
                               ('lin',     LinearRegression())
                              ])
parameters = {'lin__fit_intercept':[True,False], 
              'lin__normalize'    :[True,False], 
              'lin__copy_X'       :[True,False]}
grid = GridSearchCV(estimator =mlpipeline, 
                    param_grid=parameters, 
                    cv        =KFold(n_splits=3, shuffle=True))
grid.fit(ais_t_df, import_t_y)
y_pred=grid.predict(ais_testnew_df)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print("MSE :", mean_squared_error(import_test_y, 
                                   y_pred))

MSE : 32858959.970493697

Creating the prediction of the testing data using the `mlpipeline` defined in the cell above. Fit the pipeline to the training data and create predictions from the test dataset.

In [11]:
import_tst_predict = mlpipeline.fit(ais_train_df,
                                  import_t_y)\
                                .predict(ais_test_df)

Create the test target value `import_test_y`

In [13]:
import_test_y = np.array(ais_testnew_df.Average.shift(-5))
where_nan1=np.isnan(import_test_y)
import_test_y[where_nan1]=0

Here we are using the zip command to put the y values and predictions of the testing data side by side. Thus, enabling us to compare the values rather easily.

In [15]:
list(zip(import_test_y, 
         import_tst_predict))

Out[ 53 ]: 
[(3594.0, -14357.602199411907),
 (3261.0, -16832.49853720478),
 (3101.0, -16794.801940118094),
 (2941.0, -16972.603226673178),
 (2748.0, -16187.749616244255),
 (2730.0, -15780.369309873888),
 (2662.0, -15557.778892598973),
 (2706.0, -14675.91092958617),
 (2808.0, -14432.019469263498),
 (2984.0, -15657.733623262262),
 (3184.0, -4919.707605555199),
 (3118.0, -6700.603539934487),
 (3063.0, -7086.067551687931),
 (2851.0, -7316.985128472817),
 (2785.0, -6800.907317180292),
 (2806.0, -7451.168388386821),
 (2822.0, -7504.250690383826),
 (2775.0, -6286.542445574061),
 (2743.0, -7193.705761487017),
 (2824.0, -7150.918640656168),
 (2781.0, -7776.570830212815),
 (2771.0, -7220.143093000788),
 (2776.0, -7652.997438184673),
 (2756.0, -7565.509700210812),
 (2698.0, -8256.112466649673),
 (2721.0, -8301.834799325672),
 (2805.0, -8232.549359364722),
 (2799.0, -8346.69858020666),
 (2772.0, -8033.180253096023),
 (2642.0, -7569.421987141297),
 (2578.0, -4932.664703582141),
 (2507.0, -5449.46197421844),
 (2477.0, -5565.2765293931625),
 (2457.0, -6626.852734630895),
 (2396.0, -6854.03586406785),
 (2356.0, -7115.804524364332),
 (2327.0, -7682.903272800035),
 (2291.0, -7702.56115959051),
 (2236.0, -7658.134441776248),
 (2221.0, -7447.58002559924),
 (2185.0, -7109.321051843155),
 (2286.0, -8565.072088072771),
 (2252.0, -8158.644359325135),
 (2218.0, -8569.45062848824),
 (2172.0, -8359.770877701721),
 (2158.0, -9988.811480095275),
 (2211.0, -9009.770669820398),
 (2034.0, -9013.589337465353),
 (1985.0, -9315.590446361559),
 (2036.0, -9076.549229457029),
 (2031.0, -8050.920434263295),
 (1994.0, -8599.19371711923),
 (2005.0, -8549.57757514877),
 (2082.0, -8376.343280107372),
 (2099.0, -9120.625861667737),
 (2151.0, -9585.67563339119),
 (2518.0, -9424.763793467195),
 (3013.0, -9898.467070873448),
 (3512.0, -9772.56953562863),
 (3840.0, -9005.37255271343),
 (4027.0, -8944.678855338338),
 (4533.0, -8785.637121138694),
 (5089.0, -8428.82918586937),
 (5533.0, -7456.469334195244),
 (5537.0, -7981.512888150297),
 (5515.0, -8391.312609975204),
 (6184.0, -8167.433952201369),
 (7076.0, -6690.197461502372),
 (7739.0, -5948.716378002784),
 (8024.0, -5628.402228172075),
 (7745.0, -5196.84772705365),
 (7639.0, -4194.806432114714),
 (8216.0, -3879.243797831641),
 (8276.0, -3826.839381559821),
 (8789.0, -3590.9526548962276),
 (9170.0, -4401.861496860642),
 (9050.0, -4849.4321045596225),
 (8891.0, -3962.4277986333327),
 (8309.0, -4343.847746372885),
 (7418.0, -3361.207603181734),
 (7175.0, -2474.233153804431),
 (7024.0, -3026.8155736630506),
 (6613.0, -3548.282141456155),
 (5946.0, -5229.329664409204),
 (5521.0, -6045.3807979109115),
 (5552.0, -6815.340456667298),
 (6286.0, -7589.1269496760215),
 (6740.0, -7371.804067530284),
 (7791.0, -8339.357934710984),
 (7729.0, -8541.526582672719),
 (7378.0, -7958.987679184014),
 (6981.0, -6553.022827778623),
 (6946.0, -6268.3544935310565),
 (6726.0, -4730.15880197801),
 (6281.0, -5977.232299116316),
 (6175.0, -6506.986585276922),
 (6346.0, -7005.389523292462),
 (6538.0, -7262.163127507651),
 (6541.0, -7357.873645151041),
 (6438.0, -7645.016004793099),
 (6754.0, -7366.435130911639),
 (6789.0, -6526.729624317668),
 (6769.0, -7001.390196096989),
 (7054.0, -6631.310944013567),
 (7164.0, -6206.8420248023685),
 (7241.0, -5962.458291250376),
 (7274.0, -6990.957179960205),
 (7329.0, -7298.281904058647),
 (7170.0, -5985.702697751149),
 (6938.0, -5905.001826781103),
 (6533.0, -6038.1916029051645),
 (6315.0, -4842.078464021652),
 (6108.0, -4573.955674279619),
 (6107.0, -5350.77164221718),
 (6253.0, -5499.334957874882),
 (6452.0, -5938.802102061447),
 (6491.0, -7125.151878732831),
 (6595.0, -7574.571538654382),
 (6784.0, -7320.771722417856),
 (7110.0, -7608.099270746483),
 (7353.0, -7254.85408435604),
 (7490.0, -6156.910613082502),
 (7386.0, -6086.4695298324),
 (7265.0, -6063.78512273512),
 (7188.0, -5697.737604790214),
 (7149.0, -6187.290910574942),
 (7085.0, -3282.776097487891),
 (7044.0, -5909.113794777681),
 (7369.0, -6607.13338

In the command below, we are using the `sklearn.metrics` package to calculate the Mean Squared Error, Root Mean Squared Error, the Mean Absolute Error and the Coefficient of Determination which calculates the amount of variation in the prediction that is explained by the variables in the model.

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print("MSE :", mean_squared_error(import_test_y, 
                                 import_tst_predict))
print("RMSE:", np.sqrt(mean_squared_error(import_test_y, 
                                          import_tst_predict)))
print("MAE :", mean_absolute_error(import_test_y, 
                                   import_tst_predict))
print("R2  :", r2_score(import_test_y,
                        import_tst_predict))

MSE : 155991257.69344714
RMSE: 12489.646019541433
MAE : 12195.322537947211
R2 : -8.077072849479487

As there is never enough data to train your model, removing a part of it for validation poses a problem of underfitting. By reducing the training data, we risk losing important patterns/ trends in data set, which in turn increases error induced by bias. So, what we require is a method that provides ample data for training the model and also leaves ample data for validation. K Fold cross validation does exactly that which we have performed on our decision tree model. 

In Cross Validation, data is divided into k subsets. such that each time, one of the k subsets is used as the test set/ validation set and the other k-1 subsets are put together to form a training set. The error estimation is averaged over all k trials to get total effectiveness of our model.This significantly reduces bias as we are using most of the data for fitting, and also significantly reduces variance as most of the data is also being used in validation set

In [19]:
from sklearn.model_selection import cross_val_score, KFold
scores = cross_val_score(mlpipeline, 
                         ais_t_df, 
                         import_t_y,
                         scoring="neg_mean_squared_error", 
                         cv=KFold(n_splits=3, shuffle=True))
rmse_scores = np.sqrt(-scores)
rmse_scores

Out[ 31 ]: array([2877.08102811, 5101.836533 , 2761.87305251])

Grid Search

In the next cell we are using the `sklearn.model_selection` package to define / create the hyper-parameter dictionary which we will be using in our grid analysis. Hyperparameter is basically a parameter whose value is set before the learning process even begins.

In [22]:
parameters = {'lin__fit_intercept':[True,False], 
              'lin__normalize'    :[True,False], 
              'lin__copy_X'       :[True,False]}


In the cell below, we are creating a grid from the estimator pipeline and the hyper-parameter dictionary definied in the command above.

In [24]:
grid = GridSearchCV(estimator =mlpipeline, 
                    param_grid=parameters, 
                    cv        =KFold(n_splits=3, shuffle=True))
grid

Out[ 39 ]: 
GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
 error_score='raise',
 estimator=Pipeline(steps=[('Date', DateTransformer(col_name='Average')), ('Unique', UniqueTransformer(my_var=False)), ('AppendYearMonthDay', AddDatesTransformer(my_var=False)), ('MergeFeatures', MergeFeaturesTransformer(df_name= Date DJIA
0 2012-01-03 12397.38
1 2012-01-04 12418.42
2 2012-01-..._var=False)), ('lin', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))]),
 fit_params={}, iid=True, n_jobs=1,
 param_grid={'lin__fit_intercept': [True, False], 'lin__copy_X': [True, False], 'lin__normalize': [True, False]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
 scoring=None, verbose=0)

In the cell below we are fitting the `grid`to the training dataframe and training target dataframe.

In [26]:
grid.fit(ais_t_df, 
         import_t_y)

Out[ 40 ]: 
GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
 error_score='raise',
 estimator=Pipeline(steps=[('Date', DateTransformer(col_name='Average')), ('Unique', UniqueTransformer(my_var=False)), ('AppendYearMonthDay', AddDatesTransformer(my_var=False)), ('MergeFeatures', MergeFeaturesTransformer(df_name= Date DJIA
0 2012-01-03 12397.38
1 2012-01-04 12418.42
2 2012-01-..._var=False)), ('lin', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))]),
 fit_params={}, iid=True, n_jobs=1,
 param_grid={'lin__fit_intercept': [True, False], 'lin__copy_X': [True, False], 'lin__normalize': [True, False]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
 scoring=None, verbose=0)

Here we are gathering and evaluating the best hyper-parameters selected by the grid search process.

In [28]:
grid.best_params_ 

Out[ 41 ]: {'lin__copy_X': True, 'lin__normalize': False, 'lin__fit_intercept': True}

Here we are printing the coefficient of determination (R sq) and also creating a list of the y and predicted target numbers for swift comparison of our model results.

In [30]:
print("r squared / variance   : ", grid.best_score_)

np.round(list(zip(import_test_y, 
                  grid.predict(ais_testnew_df))),
         decimals=3
        )[:100,:]

r squared / variance : 0.8390252144129577
 Out[ 44 ]: 
array([[ 3594. , -1617.845],
 [ 3261. , -2279.233],
 [ 3101. , -2618.531],
 [ 2941. , -3270.433],
 [ 2748. , -2691.604],
 [ 2730. , -2267.493],
 [ 2662. , -1983.477],
 [ 2706. , -1441.477],
 [ 2808. , -1535.65 ],
 [ 2984. , -2002.751],
 [ 3184. , 1729.935],
 [ 3118. , 260.468],
 [ 3063. , 30.558],
 [ 2851. , -248.049],
 [ 2785. , 159.007],
 [ 2806. , 131.152],
 [ 2822. , 352.701],
 [ 2775. , 907.435],
 [ 2743. , 747.017],
 [ 2824. , 461.258],
 [ 2781. , 568.054],
 [ 2771. , 1136.004],
 [ 2776. , 1111.457],
 [ 2756. , 1156.939],
 [ 2698. , 644.021],
 [ 2721. , 198.789],
 [ 2805. , 420.82 ],
 [ 2799. , 253.185],
 [ 2772. , 598.284],
 [ 2642. , 1075.294],
 [ 2578. , 2803.186],
 [ 2507. , 2431.329],
 [ 2477. , 2360.57 ],
 [ 2457. , 1574.256],
 [ 2396. , 1446.001],
 [ 2356. , 1188.379],
 [ 2327. , 685.804],
 [ 2291. , 564.405],
 [ 2236. , 621.135],
 [ 2221. , 947.03 ],
 [ 2185. , 1604.433],
 [ 2286. , -257.513],
 [ 2252. , 274.788],
 [ 2218. , 213.178],
 [ 2172. , 308.306],
 [ 2158. , -1355.098],
 [ 2211. , -644.318],
 [ 2034. , -423.559],
 [ 1985. , -691.009],
 [ 2036. , -357.7 ],
 [ 2031. , 535.237],
 [ 1994. , 65.492],
 [ 2005. , 303.796],
 [ 2082. , 410.464],
 [ 2099. , -148.166],
 [ 2151. , -571.494],
 [ 2518. , -319.385],
 [ 3013. , -753.185],
 [ 3512. , -776.006],
 [ 3840. , 385.744],
 [ 4027. , 262.644],
 [ 4533. , 272.765],
 [ 5089. , -232.153],
 [ 5533. , 947.453],
 [ 5537. , 637.776],
 [ 5515. , 217.528],
 [ 6184. , 112.384],
 [ 7076. , 1286.664],
 [ 7739. , 2134.245],
 [ 8024. , 2607.233],
 [ 7745. , 3166.614],
 [ 7639. , 3703.3 ],
 [ 8216. , 3931.152],
 [ 8276. , 4184.849],
 [ 8789. , 4544.714],
 [ 9170. , 4125.991],
 [ 9050. , 3694.48 ],
 [ 8891. , 4468.644],
 [ 8309. , 4183.503],
 [ 7418. , 4978.388],
 [ 7175. , 6024.519],
 [ 7024. , 5987.48 ],
 [ 6613. , 5705.485],
 [ 5946. , 4684.847],
 [ 5521. , 4025.674],
 [ 5552. , 3261.025],
 [ 6286. , 2534.88 ],
 [ 6740. , 2564.696],
 [ 7791. , 1817.877],
 [ 7729. , 1838.23 ],
 [ 7378. , 2346.452],
 [ 6981. , 3467.939],
 [ 6946. , 3665.795],
 [ 6726. , 4537.163],
 [ 6281. , 3760.692],
 [ 6175. , 3476.691],
 [ 6346. , 2963.362],
 [ 6538. , 2553.358],
 [ 6541. , 2307.623],
 [ 6438. , 2022.951]])

End of Notebook